In [393]:
from homeassistant.const import Platform
import pandas as pd
import numpy as np
import os
import json

In [394]:
integrations = [x for x in os.listdir("homeassistant/components") if os.path.isdir(f"homeassistant/components/{x}") and not x.startswith("__")]
df = pd.DataFrame({"domain": integrations})
df = df.sort_values("domain")
df.reset_index(inplace=True)
df = df.drop(columns=["index"])
df.head()

,domain
0,3_day_blinds
1,abode
2,accuweather
3,acer_projector
4,acmeda


In [395]:
df["name"] = None
df["quality_scale"] = None
df["iot_class"] = None
df["integration_type"] = None
df["config_flow"] = False
df["codeowners"] = 0
df["external_library"] = False
df["bluetooth"] = False
df["zeroconf"] = False
df["ssdp"] = False
df["dhcp"] = False
df["application_credentials"] = False

for integration_id, integration in df.iterrows():
    domain = integration["domain"]
    with open(f"homeassistant/components/{domain}/manifest.json") as manifest_file:
        manifest = json.load(manifest_file)
        df.at[integration_id, "name"] = manifest.get("name")
        df.at[integration_id, "quality_scale"] = manifest.get("quality_scale")
        df.at[integration_id, "external_library"] = "requirements" in manifest
        df.at[integration_id, "codeowners"] = len(manifest.get("codeowners", []))
        df.at[integration_id, "integration_type"] = manifest.get("integration_type")
        df.at[integration_id, "iot_class"] = manifest.get("iot_class")
        df.at[integration_id, "bluetooth"] = "bluetooth" in manifest
        df.at[integration_id, "zeroconf"] = "zeroconf" in manifest
        df.at[integration_id, "ssdp"] = "ssdp" in manifest
        df.at[integration_id, "dhcp"] = "dhcp" in manifest
        df.at[integration_id, "application_credentials"] = "application_credentials" in manifest.get("dependencies", [])
        df.at[integration_id, "config_flow"] = "config_flow" in manifest
df.head()

,domain,name,quality_scale,iot_class,integration_type,config_flow,codeowners,external_library,bluetooth,zeroconf,ssdp,dhcp,application_credentials
0,3_day_blinds,3 Day Blinds,None,None,virtual,False,0,False,False,False,False,False,False
1,abode,Abode,None,cloud_push,None,True,1,True,False,False,False,False,False
2,accuweather,AccuWeather,platinum,cloud_polling,service,True,1,True,False,False,False,False,False
3,acer_projector,Acer Projector,None,local_polling,None,False,0,True,False,False,False,False,False
4,acmeda,Rollease Acmeda Automate,None,local_push,None,True,1,True,False,False,False,False,False


In [396]:
df = df[df["integration_type"].isin([None, "service", "device", "hub"])]
df.head()

,domain,name,quality_scale,iot_class,integration_type,config_flow,codeowners,external_library,bluetooth,zeroconf,ssdp,dhcp,application_credentials
1,abode,Abode,None,cloud_push,None,True,1,True,False,False,False,False,False
2,accuweather,AccuWeather,platinum,cloud_polling,service,True,1,True,False,False,False,False,False
3,acer_projector,Acer Projector,None,local_polling,None,False,0,True,False,False,False,False,False
4,acmeda,Rollease Acmeda Automate,None,local_push,None,True,1,True,False,False,False,False,False
6,actiontec,Actiontec,None,local_polling,None,False,0,False,False,False,False,False,False


In [397]:
df["runtime_data"] = False
df["has_entity_name"] = False
df["device_info"] = False
df["entity_category"] = False
df["diagnostics"] = False
for integration_id, integration in df.iterrows():
    domain = integration["domain"]
    for feature in Platform:
        if os.path.isfile(f"homeassistant/components/{domain}/{feature}.py"):
            with open(f"homeassistant/components/{domain}/{feature}.py") as file:
                if "entry.runtime_data" in file.read():
                    df.at[integration_id, "runtime_data"] = True
                    break
    files = [x for x in os.listdir(f"homeassistant/components/{domain}") if os.path.isfile(f"homeassistant/components/{domain}/{x}") and x.endswith(".py")]
    if "diagnostics.py" in files:
        df.at[integration_id, "diagnostics"] = True
    for file in files:
        with open(f"homeassistant/components/{domain}/{file}") as file2:
            body = file2.read()
            if "has_entity_name = True" in body:
                df.at[integration_id, "has_entity_name"] = True
            if "DeviceInfo" in body:
                df.at[integration_id, "device_info"] = True
            if "EntityCategory" in body:
                df.at[integration_id, "entity_category"] = True
            


In [398]:
df["entity_translations"] = False
df["exception_translations"] = False
for integration_id, integration in df.iterrows():
    domain = integration["domain"]
    if os.path.isfile(f"homeassistant/components/{domain}/strings.json"):
        file = json.load(open(f"homeassistant/components/{domain}/strings.json"))
        if "entity" in file:
            df.at[integration_id, "entity_translations"] = True
        if "exceptions" in file:
            df.at[integration_id, "exception_translations"] = True

In [399]:
df["unique_id"] = False
df["abort_entries_match"] = False
df["reauth"] = False
df["reconfigure"] = False
for integration_id, integration in df.iterrows():
    domain = integration["domain"]
    if os.path.isfile(f"homeassistant/components/{domain}/config_flow.py"):
        with open(f"homeassistant/components/{domain}/config_flow.py") as file:
            body = file.read()
            if "self.async_set_unique_id" in body:
                df.at[integration_id, "unique_id"] = True
            if "self._async_abort_entries_match" in body:
                df.at[integration_id, "abort_entries_match"] = True
            if "async_step_reauth" in body:
                df.at[integration_id, "reauth"] = True
            if "async_step_reconfigure" in body:
                df.at[integration_id, "reconfigure"] = True

In [400]:
df["entry_unload"] = False
for integration_id, integration in df.iterrows():
    domain = integration["domain"]
    if os.path.isfile(f"homeassistant/components/{domain}/config_flow.py"):
        with open(f"homeassistant/components/{domain}/__init__.py") as file:
            body = file.read()
            if "async_unload_entry" in body:
                df.at[integration_id, "entry_unload"] = True

In [401]:
silver = df[df["quality_scale"] == "silver"]
gold = df[df["quality_scale"] == "gold"]
platinum = df[df["quality_scale"] == "platinum"]
up_to_gold = pd.concat([silver, gold])
up_to_platinum = pd.concat([silver, gold, platinum])
config_flow_integrations = df[df["config_flow"]]

In [402]:
def percentage(dataframe: pd.DataFrame, series: pd.Series) -> str:
    total_amount = len(dataframe)
    amount = len(series)
    return f"Total: {total_amount}, Amount: {amount}, Percentage: {amount / total_amount * 100:.2f}%"

In [403]:
percentage(df, df[df["config_flow"]])

'Total: 1054, Amount: 631, Percentage: 59.87%'

In [404]:
percentage(up_to_platinum, up_to_platinum[up_to_platinum["config_flow"]])

'Total: 94, Amount: 93, Percentage: 98.94%'

In [405]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["runtime_data"]])

'Total: 631, Amount: 106, Percentage: 16.80%'

In [406]:
percentage(platinum[platinum["config_flow"]], platinum[platinum["runtime_data"]])

'Total: 62, Amount: 27, Percentage: 43.55%'

In [407]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["abort_entries_match"]])

'Total: 93, Amount: 19, Percentage: 20.43%'

In [408]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["abort_entries_match"]])

'Total: 631, Amount: 126, Percentage: 19.97%'

In [409]:
no_abort = config_flow_integrations[config_flow_integrations["abort_entries_match"] == False]
no_abort = no_abort[no_abort["unique_id"] == False]
len(no_abort)

135

In [410]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["entry_unload"]])

'Total: 93, Amount: 93, Percentage: 100.00%'

In [411]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["has_entity_name"]])

'Total: 631, Amount: 479, Percentage: 75.91%'

In [412]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["has_entity_name"]])

'Total: 93, Amount: 81, Percentage: 87.10%'

In [413]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["entity_translations"]])

'Total: 631, Amount: 350, Percentage: 55.47%'

In [414]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["entity_translations"]])

'Total: 93, Amount: 64, Percentage: 68.82%'

In [415]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["device_info"]])

'Total: 631, Amount: 537, Percentage: 85.10%'

In [416]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["device_info"]])

'Total: 93, Amount: 89, Percentage: 95.70%'

In [417]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["entity_category"]])

'Total: 631, Amount: 205, Percentage: 32.49%'

In [418]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["entity_category"]])

'Total: 93, Amount: 42, Percentage: 45.16%'

In [419]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["ssdp"]])

'Total: 631, Amount: 38, Percentage: 6.02%'

In [420]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["bluetooth"]])

'Total: 631, Amount: 39, Percentage: 6.18%'

In [421]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["zeroconf"]])

'Total: 631, Amount: 72, Percentage: 11.41%'

In [422]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["dhcp"]])

'Total: 631, Amount: 68, Percentage: 10.78%'

In [423]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["ssdp"]])

'Total: 93, Amount: 12, Percentage: 12.90%'

In [424]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["bluetooth"]])

'Total: 93, Amount: 2, Percentage: 2.15%'

In [425]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["zeroconf"]])

'Total: 93, Amount: 20, Percentage: 21.51%'

In [426]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["dhcp"]])

'Total: 93, Amount: 12, Percentage: 12.90%'

In [427]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["reauth"]])

'Total: 631, Amount: 181, Percentage: 28.68%'

In [428]:
percentage(up_to_platinum[up_to_platinum["config_flow"]], up_to_platinum[up_to_platinum["reauth"]])

'Total: 93, Amount: 39, Percentage: 41.94%'

In [429]:
percentage(df, df[df["codeowners"] > 0])

'Total: 1054, Amount: 747, Percentage: 70.87%'

In [430]:
percentage(up_to_platinum, up_to_platinum[up_to_platinum["codeowners"] > 0])

'Total: 94, Amount: 93, Percentage: 98.94%'

In [431]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["diagnostics"]])

'Total: 631, Amount: 180, Percentage: 28.53%'

In [432]:
percentage(up_to_platinum, up_to_platinum[up_to_platinum["diagnostics"]])

'Total: 94, Amount: 63, Percentage: 67.02%'

In [433]:
percentage(config_flow_integrations, config_flow_integrations[config_flow_integrations["reconfigure"]])

'Total: 631, Amount: 13, Percentage: 2.06%'

In [434]:
percentage(up_to_platinum, up_to_platinum[up_to_platinum["reconfigure"]])

'Total: 94, Amount: 4, Percentage: 4.26%'